## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [143]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

In [144]:
pd.__version__

'1.2.0'

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * estado_vacunacion_.csv , última versión del mismo con el acumulado de todos los informes publicados
  * estado_vacunacion_year-month-day.csv, con la versión del día en cuestión, solo con la información de ese día

In [145]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
fecha_informe = datetime(year=2021,month=1,day=7)
fecha_informe_str=fecha_informe.strftime('%Y%m%d')
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en formato ods

In [146]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [147]:
datos_row = pd.read_excel(enlace)

In [148]:
datos_row

,Unnamed: 0,Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Fecha de la última vacuna registrada (2)
0,Andalucía,140295,53934,0.384433,2021-01-06
1,Aragón,23715,6186,0.260848,2021-01-06
2,Asturias,23720,14406,0.607336,2021-01-06
3,Baleares,12840,3190,0.248442,2021-01-05
4,Canarias,27660,7330,0.265004,2021-01-06
5,Cantabria,11850,1564,0.131983,2021-01-07
6,Castilla y Leon,51390,22103,0.430103,2021-01-06
7,Castilla La Mancha,35550,7666,0.215640,2021-01-06
8,Cataluña,120545,24515,0.203368,2021-01-06
9,C. Valenciana,61225,11677,0.190723,2021-01-07


..copia para tratarlo

In [149]:
datos_hoy = datos_row.copy()

renombramos columnas, añadimos columna con `fecha_de_informe` y ordenamos

In [150]:
datos_hoy.columns=['ccaa','Dosis entregadas','Dosis administradas','% sobre entregadas','Fecha de la ultima vacuna registrada']

In [151]:
datos_hoy['date_pub']=fecha_informe
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace

In [152]:
datos_hoy=datos_hoy[['date_pub','ccaa','Fecha de la ultima vacuna registrada','Dosis entregadas','Dosis administradas','% sobre entregadas','source_name','source']]

eliminamos la fila Totales

In [153]:
datos_hoy = datos_hoy.drop(datos_hoy.index[-1])

In [154]:
datos_hoy

,date_pub,ccaa,Fecha de la ultima vacuna registrada,Dosis entregadas,Dosis administradas,% sobre entregadas,source_name,source
0,2021-01-07,Andalucía,2021-01-06,140295,53934,0.384433,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,2021-01-07,Aragón,2021-01-06,23715,6186,0.260848,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,2021-01-07,Asturias,2021-01-06,23720,14406,0.607336,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,2021-01-07,Baleares,2021-01-05,12840,3190,0.248442,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,2021-01-07,Canarias,2021-01-06,27660,7330,0.265004,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,2021-01-07,Cantabria,2021-01-07,11850,1564,0.131983,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,2021-01-07,Castilla y Leon,2021-01-06,51390,22103,0.430103,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,2021-01-07,Castilla La Mancha,2021-01-06,35550,7666,0.215640,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,2021-01-07,Cataluña,2021-01-06,120545,24515,0.203368,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,2021-01-07,C. Valenciana,2021-01-07,61225,11677,0.190723,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [155]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [156]:
datos_acumulados = pd.read_csv('csv/Libro1.csv',sep=';')

In [157]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [158]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,2021-01-04,Andalucía,140295,25809,"18,40%",3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,2021-01-04,Aragón,23715,2004,"8,50%",2/1/21,2021-01-02,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,2021-01-04,Asturias,23720,9380,"39,50%",3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,2021-01-04,Baleares,8940,153,"1,70%",28/12/20,2020-12-28,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,2021-01-04,Canarias,20835,4846,"23,25%",3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,2021-01-04,Cantabria,11850,304,"2,60%",30/12/20,2020-12-30,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,2021-01-04,Castilla y Leon,51390,10928,"21,26%",3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,2021-01-04,Castilla La Mancha,35550,1983,"5,60%",3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,2021-01-04,Cataluña,120545,8150,"6,80%",3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,2021-01-04,C. Valenciana,61225,3252,"5,30%",3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [140]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [139]:
#aseguramos formato fecha para evitar inconsistencia en el merge
datos_acumulados['date_pub'] = pd.to_datetime(datos_acumulados['date_pub'],format='%d/%m/%y')
datos_acumulados['Fecha de la ultima vacuna registrada'] = pd.to_datetime(datos_acumulados['Fecha de la ultima vacuna registrada'],format='%d/%m/%y')

In [159]:
datos_acumulados['% sobre entregadas']=datos_acumulados['Dosis administradas']/datos_acumulados['Dosis entregadas']

In [160]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,2021-01-04,Andalucía,140295,25809,0.183962,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,2021-01-04,Aragón,23715,2004,0.084503,2/1/21,2021-01-02,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,2021-01-04,Asturias,23720,9380,0.395447,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,2021-01-04,Baleares,8940,153,0.017114,28/12/20,2020-12-28,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,2021-01-04,Canarias,20835,4846,0.232589,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,2021-01-04,Cantabria,11850,304,0.025654,30/12/20,2020-12-30,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,2021-01-04,Castilla y Leon,51390,10928,0.212648,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,2021-01-04,Castilla La Mancha,35550,1983,0.055781,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,2021-01-04,Cataluña,120545,8150,0.067610,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,2021-01-04,C. Valenciana,61225,3252,0.053116,3/1/21,2021-01-03,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


y volvemos a subirlo..

In [162]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)